# Activity Investigation

This should be helpful when I just want to look closely at the results of a particular activity stream

In [1]:
from strava.data.cache import Cache
from strava.data.strava_requests import *
from strava.plotting.strava_annual_plots import *
from strava.plotting.strava_stream_plots import *
from generate_plots import get_clean_activities, get_clean_stream
from strava.data.cache import Cache

import aiohttp
import asyncio
import copy
from datetime import date
import datetime
import json
import matplotlib.pyplot as plt
import numpy as np
import requests
import os
import pandas as pd
from pandas_profiling import ProfileReport
from plotnine import ggplot, aes
import plotnine as gg
import time
from typing import List
import urllib3
import PIL
from PIL import Image
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
gg.options.figure_size = (11, 8.5)
pt.dark()

pd.set_option('display.max_columns', 500)

%load_ext autoreload
%autoreload 2

In [2]:
# Create a cache
stream_cache = Cache()
activity_cache = Cache(dir='/Users/lucasnieuwenhout/Documents/Programming/Python/Projects/StravaPlotting/activity_cache/')

In [3]:
# Get a list of activities
activities = get_clean_activities()
activities.tail()

Existing access token is valid.
Beginning request of activities...
-> Received page 1 with 200 activities.
-> Received page 2 with 200 activities.
-> Received page 3 with 200 activities.
-> Received page 4 with 40 activities.
-> Received page 5 with 0 activities.
...retrieved total of 640 activities.


,id,name,distance,moving_time,elapsed_time,total_elevation_gain,elev_high,elev_low,sport_type,start_date_local,timezone,start_latlng,end_latlng,achievement_count,map.id,workout_type,average_speed,max_speed,average_heartrate,max_heartrate,map.summary_polyline,year,date_year_agnostic
634,3425015993,Afternoon Ride,8964.6,0 days 00:26:43,0 days 00:26:43,39.0,693.8,654.7,Ride,2020-04-26 15:07:20,(GMT-07:00) America/Edmonton,"[53.636688, -113.587822]","[53.636729, -113.587903]",0,a3425015993,10.0,20.1312,40.32,165.0,185.0,{mjfIhhjtTt@fAf@|@HREZUpAe@pB{@vC]z@a@r@k@x@c@...,2020,1970-04-26 15:07:20
635,3425016238,Lunch Ride,17516.8,0 days 00:48:11,0 days 00:48:11,71.2,693.8,653.7,Ride,2020-04-25 11:48:43,(GMT-07:00) America/Edmonton,"[53.636724, -113.587843]","[53.636742, -113.587806]",0,a3425016238,10.0,21.8124,50.04,178.0,194.0,knjfIrhjtT`AxAb@v@G|@UvA[pAw@fCy@hBg@t@g@l@u@n...,2020,1970-04-25 11:48:43
636,3425017204,Afternoon Ride,14517.0,0 days 00:47:28,0 days 00:47:28,64.1,693.8,653.7,Ride,2020-04-23 13:02:13,(GMT-07:00) America/Edmonton,"[53.636938, -113.587945]","[53.636747, -113.58785]",0,a3425017204,10.0,18.3492,36.72,175.4,198.0,qnjfIlhjtTfAvAl@bAEp@OdA]hBc@lBW~@]~@Yj@g@t@_@...,2020,1970-04-23 13:02:13
637,3425016478,Afternoon Ride,17605.8,0 days 00:57:41,0 days 00:57:41,69.9,693.8,653.7,Ride,2020-04-22 14:48:16,(GMT-07:00) America/Edmonton,"[53.636929, -113.587659]","[53.636694, -113.587956]",0,a3425016478,10.0,18.3132,39.96,176.6,194.0,wmjfIdhjtTx@pAZv@F^?XAb@EZUt@[r@qBpCw@rA_@b@s@...,2020,1970-04-22 14:48:16
639,3425016348,Afternoon Ride,7278.0,0 days 00:23:28,0 days 00:23:28,21.9,693.8,684.9,Ride,2020-04-16 13:57:58,(GMT-07:00) America/Edmonton,"[53.636894, -113.587782]","[53.636646, -113.587864]",0,a3425016348,10.0,18.6084,36.00,177.8,197.0,uojfIxhjtTvB|CjAnBbApBlA|CXdC^tEL`CB|ACtBKjBs@...,2020,1970-04-16 13:57:58


# Explore Activity API

What do I actually want to do?
- Plot my progress on segments of interest
- Plot how many of the 37 segments I have local legend on
- Get a list of the canonical segments and find my time vs. the best time?
- 

In [50]:
activities

,id,name,distance,moving_time,elapsed_time,total_elevation_gain,elev_high,elev_low,sport_type,start_date_local,timezone,start_latlng,end_latlng,achievement_count,map.id,workout_type,average_speed,max_speed,average_heartrate,max_heartrate,map.summary_polyline,year,date_year_agnostic
0,14296525748,Afternoon Ride,28427.8,0 days 01:10:45,0 days 01:13:04,352.3,100.6,3.4,Ride,2025-04-26 16:21:07,(GMT-08:00) America/Vancouver,"[49.055061, -122.774509]","[49.055059, -122.774641]",6,a14296525748,NaN,24.1092,50.328,171.3,189.0,w`ljHhtjlVZSZCLSFBA|@DN\l@LtAt@bDJtAf@v@FrAPt@...,2025,1970-04-26 16:21:07
1,14277777811,Afternoon Ride,28439.2,0 days 01:12:05,0 days 01:14:49,352.4,100.6,3.4,Ride,2025-04-24 16:42:09,(GMT-08:00) America/Vancouver,"[49.055104, -122.774563]","[49.055067, -122.774657]",2,a14277777811,NaN,23.6736,60.408,175.6,190.0,u`ljHptjlVb@YPCPYDPAt@b@dANzAn@dCNhB\`@HVFrANl...,2025,1970-04-24 16:42:09
3,14246648380,Afternoon Ride,28549.4,0 days 01:13:12,0 days 01:15:12,354.3,100.6,3.4,Ride,2025-04-21 13:54:15,(GMT-08:00) America/Vancouver,"[49.055032, -122.774484]","[49.055061, -122.774616]",8,a14246648380,10.0,23.4000,58.536,171.6,193.0,u`ljHjtjlVb@YN?NSHFCR@f@d@z@HjAt@`DJvAFRd@x@Bh...,2025,1970-04-21 13:54:15
4,14218149073,Afternoon Ride,28507.2,0 days 01:17:43,0 days 01:19:52,352.0,100.6,3.4,Ride,2025-04-18 16:41:11,(GMT-08:00) America/Vancouver,"[49.055033, -122.7746]","[49.055058, -122.77465]",4,a14218149073,10.0,22.0068,59.184,167.7,189.0,e_ljHpsjlVPUDFDbA`@x@JtAn@hCPhBh@x@F`BPh@LHjCG...,2025,1970-04-18 16:41:11
5,14208696270,Afternoon Ride,28463.9,0 days 01:11:30,0 days 01:13:01,352.2,100.6,3.4,Ride,2025-04-17 16:51:20,(GMT-08:00) America/Vancouver,"[49.055069, -122.774553]","[49.055062, -122.774649]",1,a14208696270,10.0,23.8860,59.688,169.6,185.0,w`ljHltjlVH?VWRAPWHDCb@Dh@^x@N`Br@rCFfA\b@JVFz...,2025,1970-04-17 16:51:20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634,3425015993,Afternoon Ride,8964.6,0 days 00:26:43,0 days 00:26:43,39.0,693.8,654.7,Ride,2020-04-26 15:07:20,(GMT-07:00) America/Edmonton,"[53.636688, -113.587822]","[53.636729, -113.587903]",0,a3425015993,10.0,20.1312,40.320,165.0,185.0,{mjfIhhjtTt@fAf@|@HREZUpAe@pB{@vC]z@a@r@k@x@c@...,2020,1970-04-26 15:07:20
635,3425016238,Lunch Ride,17516.8,0 days 00:48:11,0 days 00:48:11,71.2,693.8,653.7,Ride,2020-04-25 11:48:43,(GMT-07:00) America/Edmonton,"[53.636724, -113.587843]","[53.636742, -113.587806]",0,a3425016238,10.0,21.8124,50.040,178.0,194.0,knjfIrhjtT`AxAb@v@G|@UvA[pAw@fCy@hBg@t@g@l@u@n...,2020,1970-04-25 11:48:43
636,3425017204,Afternoon Ride,14517.0,0 days 00:47:28,0 days 00:47:28,64.1,693.8,653.7,Ride,2020-04-23 13:02:13,(GMT-07:00) America/Edmonton,"[53.636938, -113.587945]","[53.636747, -113.58785]",0,a3425017204,10.0,18.3492,36.720,175.4,198.0,qnjfIlhjtTfAvAl@bAEp@OdA]hBc@lBW~@]~@Yj@g@t@_@...,2020,1970-04-23 13:02:13
637,3425016478,Afternoon Ride,17605.8,0 days 00:57:41,0 days 00:57:41,69.9,693.8,653.7,Ride,2020-04-22 14:48:16,(GMT-07:00) America/Edmonton,"[53.636929, -113.587659]","[53.636694, -113.587956]",0,a3425016478,10.0,18.3132,39.960,176.6,194.0,wmjfIdhjtTx@pAZv@F^?XAb@EZUt@[r@qBpCw@rA_@b@s@...,2020,1970-04-22 14:48:16


In [125]:
for id in list(activities.query('distance > 28000 & distance < 29000').id):
    get_activity_detail(activities, id, activity_cache, include_all_efforts=True)

Found activity 14296525748 in cache.
Found activity 14277777811 in cache.
Found activity 14246648380 in cache.
Found activity 14218149073 in cache.
Found activity 14208696270 in cache.
Found activity 14180028159 in cache.
Found activity 14169893236 in cache.
Found activity 14150724069 in cache.
Found activity 14132622357 in cache.
Found activity 14082982033 in cache.
Found activity 10033794476 in cache.
Found activity 9996006537 in cache.
Found activity 9990461310 in cache.
Found activity 9985137901 in cache.
Found activity 9978987119 in cache.
Found activity 9940528230 in cache.
Found activity 9911215580 in cache.
Found activity 9902256544 in cache.
Found activity 9851417814 in cache.
Found activity 9831605045 in cache.
Found activity 9824564333 in cache.
Found activity 9805624701 in cache.
Found activity 9765687810 in cache.
Found activity 9759475809 in cache.
Found activity 9753449004 in cache.
Found activity 9732826501 in cache.
Found activity 9721132382 in cache.
Found activity 97

In [118]:
activ_id = activity_id_by_date(activities=activities, date=datetime.date(2025, 4, 26))
activity_detail = get_activity_detail(activities, activ_id, activity_cache, include_all_efforts=True)
activity_detail

Found activity 14296525748 in cache.


{'resource_state': 3,
 'athlete': {'id': 56778178, 'resource_state': 1},
 'name': 'Afternoon Ride',
 'distance': 28427.8,
 'moving_time': 4245,
 'elapsed_time': 4384,
 'total_elevation_gain': 352.3,
 'type': 'Ride',
 'sport_type': 'Ride',
 'workout_type': None,
 'id': 14296525748,
 'start_date': '2025-04-26T23:21:07Z',
 'start_date_local': '2025-04-26T16:21:07Z',
 'timezone': '(GMT-08:00) America/Vancouver',
 'utc_offset': -25200.0,
 'location_city': None,
 'location_state': None,
 'location_country': None,
 'achievement_count': 6,
 'kudos_count': 1,
 'comment_count': 0,
 'athlete_count': 1,
 'photo_count': 0,
 'map': {'id': 'a14296525748',
  'polyline': 'caljHtkjlVIKOYKGK@GJGPEh@@`AHl@j@nBFLHHH?ZSPCH?LSD?@BA|@DN\\l@LtAt@bDJtAHNRPHTD^@r@Lj@HPF@VEjAFf@CDHD|@Ft@DjECjBDtBGpC?b]FxLE`I@PLt@Eb@?hCAj@Il@@HPT?Da@h@ADF@PWABB@HTb@|@@Fi@p@ET@HBBB?JKP[T]n@m@vBeCd@o@~BeCdAuAvDeE^[`@OVGdB@nHIdBF|A?rCFv@AtD@l@KfAo@^GZ@b@H~@f@j@VPD`@Br@Gd@IPA`H?n@C~EBbCA~GBlBCjI?vAGjA@t@DnA@xHEbC@pBInB?jCErHJ~@CxGBN@T

In [ ]:
def activity_detail_print_simple_summary(activity_detail):
    """Print simple summary of activity"""
    print(f"Date: \t\t{activity_detail['start_date_local'].split('T')[0]}")
    print(f"Name: \t\t{activity_detail['name']}")
    print(f"Distance: \t{activity_detail['distance'] / 1000:0.3f} km")
    print(f"Time: \t\t{activity_detail['moving_time'] // 3600}:{activity_detail['moving_time'] % 3600 // 60}:{activity_detail['moving_time'] % 60}")
    print(f"Elevation: \t{activity_detail['total_elevation_gain']} m")
    print(f"Achievements: \t{activity_detail['achievement_count']}")
    print(f"Average Speed: \t{activity_detail['average_speed'] / 1000 * 3600:0.2f} km/h")
    print(f"Average HR: \t{round(activity_detail['average_heartrate'])}")
    print(f"Segments: \t{len(activity_detail['segment_efforts'])}")

activity_detail_print_simple_summary(activity_detail)

Date: 		2025-04-26
Name: 		Afternoon Ride
Distance: 	28.428 km
Time: 		1:10:45
Elevation: 	352.3 m
Achievements: 	6
Average Speed: 	24.11 km/h
Average HR: 	171
Segments: 	37


In [46]:
activity_detail['segment_efforts'][0]

{'id': 3348481738355317272,
 'resource_state': 2,
 'name': '28th - it says bike route....',
 'activity': {'id': 14218149073,
  'visibility': 'everyone',
  'resource_state': 1},
 'athlete': {'id': 56778178, 'resource_state': 1},
 'elapsed_time': 111,
 'moving_time': 111,
 'start_date': '2025-04-18T23:43:15Z',
 'start_date_local': '2025-04-18T16:43:15Z',
 'distance': 734.2,
 'start_index': 123,
 'end_index': 234,
 'device_watts': False,
 'average_heartrate': 171.2,
 'max_heartrate': 176.0,
 'segment': {'id': 31582444,
  'resource_state': 2,
  'name': '28th - it says bike route....',
  'activity_type': 'Ride',
  'distance': 734.2,
  'average_grade': 2.0,
  'maximum_grade': 5.1,
  'elevation_high': 75.2,
  'elevation_low': 58.6,
  'start_latlng': [49.052882, -122.78008],
  'end_latlng': [49.052823, -122.790121],
  'elevation_profile': None,
  'elevation_profiles': None,
  'climb_category': 0,
  'city': 'Surrey',
  'state': 'British Columbia',
  'country': 'Canada',
  'private': False,
  'h

In [48]:
# id(segment effort id), name, elapsed_time, moving_time, distance, average_heartrate, max_heartrate, segment
#  - name, distance, average_grade, maximum_grade, elevation_high, elevation_low, hazardous, pr_rank, achievements
#    - type, rank, effort_count

#   'resource_state': 2,
#   'name': '28th - it says bike route....',
#   'activity_type': 'Ride',
#   'distance': 734.2,
#   'average_grade': 2.0,
#   'maximum_grade': 5.1,
#   'elevation_high': 75.2,
#   'elevation_low': 58.6,
#   'start_latlng': [49.052882, -122.78008],
#   'end_latlng': [49.052823, -122.790121],
#   'elevation_profile': None,
#   'elevation_profiles': None,
#   'climb_category': 0,
#   'city': 'Surrey',
#   'state': 'British Columbia',
#   'country': 'Canada',
#   'private': False,
#   'hazardous': False,
#   'starred': False},
#  'pr_rank': None,
#  'achievements': [{'type_id': 7,
#    'type': 'segment_effort_count_leader',
#    'rank': 1,
#    'effort_count': 10}],

# Achievements
# Achievements is a list of dictinoaries holding various things
# - type_id = 7: 'type': 'segment_effort_count_leader', 'rank': 1, 'effort_count'
# - type_id = 3: 'type': 'pr', 'rank'
# - type_id = 2: 'type': 'overall', 'rank'

seg_df = []
for seg in activity_detail['segment_efforts']:
    segment_detail = [seg['id']]
    segment_detail.append(seg['segment']['id'])
    segment_detail.append(seg['name'])
    segment_detail.append(seg['distance'])
    segment_detail.append(seg['moving_time'])
    segment_detail.append(seg['average_heartrate'])
    segment_detail.append(seg['segment']['average_grade'])
    segment_detail.append(seg['segment']['hazardous'])
    segment_detail.append(seg['pr_rank'])
    segment_detail.append(seg['achievements'])
    if len(seg['achievements']) > 0:
        print(seg['achievements'])


    seg_df.append(segment_detail)
seg_df = pd.DataFrame(seg_df)
seg_df.columns = ['Effort ID', 'Segment ID', 'Name', 'Distance[m]', 'Moving Time[s]', 'Average HR', 'Average Grade', 'Hazardous', 'PR Rank', 'Achievements']

seg_df

[{'type_id': 7, 'type': 'segment_effort_count_leader', 'rank': 1, 'effort_count': 10}]
[{'type_id': 7, 'type': 'segment_effort_count_leader', 'rank': 1, 'effort_count': 10}]
[{'type_id': 7, 'type': 'segment_effort_count_leader', 'rank': 1, 'effort_count': 10}]
[{'type_id': 3, 'type': 'pr', 'rank': 3}]
[{'type_id': 7, 'type': 'segment_effort_count_leader', 'rank': 1, 'effort_count': 10}]
[{'type_id': 7, 'type': 'segment_effort_count_leader', 'rank': 1, 'effort_count': 10}]
[{'type_id': 7, 'type': 'segment_effort_count_leader', 'rank': 1, 'effort_count': 10}]
[{'type_id': 7, 'type': 'segment_effort_count_leader', 'rank': 1, 'effort_count': 10}]
[{'type_id': 3, 'type': 'pr', 'rank': 1}, {'type_id': 7, 'type': 'segment_effort_count_leader', 'rank': 1, 'effort_count': 10}]
[{'type_id': 3, 'type': 'pr', 'rank': 1}, {'type_id': 7, 'type': 'segment_effort_count_leader', 'rank': 1, 'effort_count': 10}]
[{'type_id': 7, 'type': 'segment_effort_count_leader', 'rank': 1, 'effort_count': 10}]
[{'typ

,Effort ID,Segment ID,Name,Distance[m],Moving Time[s],Average HR,Average Grade,Hazardous,PR Rank,Achievements
0,3348481738355317272,31582444,28th - it says bike route....,734.200,111,171.2,2.0,False,NaN,"[{'type_id': 7, 'type': 'segment_effort_count_..."
1,3348481738356708888,25667954,KG to 20 on 156,474.000,70,176.1,-0.1,False,NaN,[]
2,3348481738355444248,10267456,It smells richer in White Rock,1729.200,231,180.5,0.1,False,NaN,"[{'type_id': 7, 'type': 'segment_effort_count_..."
3,3348481738357081624,20245971,The Leafs Blow,374.900,50,145.0,-4.9,False,NaN,"[{'type_id': 7, 'type': 'segment_effort_count_..."
4,3348481738356379160,1145071,Marine Drive Rollers,1888.500,585,149.2,-0.2,True,NaN,[]
5,3348481738357499416,3935337,white rockin',1847.700,574,149.4,-0.3,True,NaN,[]
6,3348481738357747224,19129655,Cruisin' East Beach - Westbound,404.600,150,140.2,0.4,False,NaN,[]
7,3348481738356374040,16233163,marine kicker,317.200,49,181.9,6.4,False,NaN,[]
8,3348481738355226136,19059308,Rollin' Restaurant Row,616.400,261,146.5,-1.1,False,NaN,[]
9,3348481738358563352,19628715,Hotties & Hot Rods,486.600,69,153.0,0.7,False,NaN,[]


# Explore Segment API
- This is quite interesting...
- I can't use the segment effort api, but I can use the segment detail one.  This hold the kom and local legend details.


- Would be good to pull this information for each segment.  Compare against my best efforts.
- Can plot average time per month or season or something, or best time per month/season.
  - Could potentially plot the heartrate for the best effort?  Or the maximum speed?

In [111]:
segment_effort_url = "https://www.strava.com/api/v3/segments/"

In [113]:
def request_segment(segment_id: int, per_page: int=30):
    """Request activity data."""

    # Set up request parameters
    header = {'Authorization': 'Bearer ' + get_access_token()}
    params = {"per_page": per_page}
    params = {}
    url = segment_effort_url + str(segment_id)

    # Request the streams
    stream_resp = requests.get(url, params=params, headers=header)
    if stream_resp.status_code != 200:
        print(f"Request failed, status code: {stream_resp.status_code}")
        return None

    print(f"Received {len(stream_resp.content)} bytes.")
    return stream_resp

In [114]:
r = request_segment(31582444)

Existing access token is valid.
Received 1769 bytes.


In [117]:
json.loads(r.content)

{'id': 31582444,
 'resource_state': 3,
 'name': '28th - it says bike route....',
 'activity_type': 'Ride',
 'distance': 734.2,
 'average_grade': 2.0,
 'maximum_grade': 5.1,
 'elevation_high': 75.2,
 'elevation_low': 58.6,
 'start_latlng': [49.052882, -122.78008],
 'end_latlng': [49.052823, -122.790121],
 'elevation_profile': 'https://d3o5xota0a1fcr.cloudfront.net/v6/charts/BRLSPYSPS7J4TCZ3QJO3W62FBPDAQRLHHAVJ4KOLRZDFLOBZOLWP5ZNDURNWZHIIESTCAFPJ5YKOLFLK5ZF3FIOO',
 'elevation_profiles': {'light_url': 'https://d3o5xota0a1fcr.cloudfront.net/v6/charts/BRLSPYSPS7J4TCZ3QJO3W62FBPDAQRLHHAVJ4KOLRZDFLOBZOLWP5ZNDURNWZHIIESTCAFPJ5YKOLFLK5ZF3FIOO',
  'dark_url': 'https://d3o5xota0a1fcr.cloudfront.net/v6/charts/SE4BUDO3W6BSEJDS43J4YZB3TTI7BEN5FJ7IFWPEWDQDFOE7TX4G2RFSCRFJU6OJZCTWRM2CXW6AXMTDT4P6XJDS'},
 'climb_category': 0,
 'city': 'Surrey',
 'state': 'British Columbia',
 'country': 'Canada',
 'private': False,
 'hazardous': False,
 'starred': False,
 'created_at': '2022-04-24T20:34:56Z',
 'updated_